In [16]:
import tensorflow as tf
import numpy as np
from tensorflow.python.framework import ops
ops.reset_default_graph()

# 데이터가 한글자씩 들어감
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀연습사랑'] 
                                
num_dic =  {n: i for i , n  in enumerate(char_arr)} #enumerate : 각 데이터에 번호를 달아줌
dic_len = len(num_dic)
print(dic_len)
seq_data =[['word','단어'],['wood','나무'],
           ['game','놀이'],['girl','소녀'],
           ['test','연습'],['love','사랑']]
#seq2seq 번역망

def make_batch(seq_data) :
    
    input_batch = []
    output_batch = []
    target_batch = []
    for seq in seq_data:
        input = [num_dic[n] for n in seq[0]]          # word => num_dic에서 번호로 바뀜 25,17,..
        
        # 첫번째 입력 데이터는 'word'이 학습된 smoking gun이 들어옴 
        output = [num_dic[n] for n in ("S" + seq[1])] # Start의 S  => 시작을 확인하기 위해서
        target = [num_dic[n] for n in (seq[1] + 'E')] # End => 끝을 확인하기 위해서( 글자 수가 다른 경우, 인덱스로 지정이 어렵기 떄문에 E를 끝으로 설정하여 출력)
        
        input_batch.append(np.eye(dic_len)[input]) # 42개 대각행렬( 단위행렬(identity matrix)) => one-hot 인코딩
        # 숫자로된 word가 one-hot되어 4x41가 만들어짐
        
        output_batch.append(np.eye(dic_len)[output])
        # 2글자이므로 one-hot되어 2x41
        
        target_batch.append(target)
    return input_batch, output_batch, target_batch

learning_rate = 0.01
n_hidden = 128    # neurons 출력이 128차
total_epoch = 100
n_class = n_input = dic_len

# encoder, decoder용 데이터 주입 변수
enc_input = tf.placeholder(tf.float32, [None,None,n_input])   
dec_input = tf.placeholder(tf.float32, [None,None,n_input])
targets = tf.placeholder(tf.int64, [None,None])
with tf.variable_scope('encode'):
    # FFNN 한개 => 입력 -> 가중치 ---> 출력(n_hidden)
    enc_cell =tf.nn.rnn_cell.BasicRNNCell(n_hidden)   # 입력 41 x 128 
    
    # 과적합 방지를 위해 계산회로 생략 - random
    
    #셀이 4개 # 셀 당 41개가 들어옴
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell,
                                            output_keep_prob = 0.5, seed=100)
    otuputs , enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype = tf.float32)

with tf.variable_scope('decode') :
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)   # 128
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell,output_keep_prob = 0.5)
    # 2 x 41
    # 앞의 인코더 망에서 출력된 states값(마지막 셀의 출력값==> 앞의 데이터를 모두 고려함(셀로 연결))

    outputs , dec_stats = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state= enc_states, dtype = tf.float32)  
    # initial_state로 인코드 망과 디코드망을 연결

# output = 6 x 3(글자2개) x 128
# n_class : 41
# => 6x 3x 41
model = tf.layers.dense(outputs, n_class, activation = None)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
                    logits = model, labels = targets))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

41


In [17]:
seq_data[1][1] + 'E'

'나무E'

In [18]:
input_batch, output_batch, target_batch = make_batch(seq_data)
for epoch in range(total_epoch) :  # 100개
    _, loss = sess.run([optimizer, cost],
                      feed_dict = {enc_input: input_batch,
                                  dec_input : output_batch,
                                  targets : target_batch})
    print('Epoch : ', '%04d' % (epoch+1),
         'cost =  {:.6f}'.format(loss))

Epoch :  0001 cost =  3.710885
Epoch :  0002 cost =  2.686655
Epoch :  0003 cost =  1.762030
Epoch :  0004 cost =  1.124454
Epoch :  0005 cost =  0.607119
Epoch :  0006 cost =  0.341465
Epoch :  0007 cost =  0.509761
Epoch :  0008 cost =  0.282352
Epoch :  0009 cost =  0.230448
Epoch :  0010 cost =  0.212430
Epoch :  0011 cost =  0.262437
Epoch :  0012 cost =  0.154575
Epoch :  0013 cost =  0.137292
Epoch :  0014 cost =  0.249427
Epoch :  0015 cost =  0.025318
Epoch :  0016 cost =  0.167988
Epoch :  0017 cost =  0.217812
Epoch :  0018 cost =  0.191831
Epoch :  0019 cost =  0.126363
Epoch :  0020 cost =  0.203667
Epoch :  0021 cost =  0.163226
Epoch :  0022 cost =  0.056943
Epoch :  0023 cost =  0.134108
Epoch :  0024 cost =  0.074068
Epoch :  0025 cost =  0.013083
Epoch :  0026 cost =  0.034597
Epoch :  0027 cost =  0.023425
Epoch :  0028 cost =  0.009269
Epoch :  0029 cost =  0.022525
Epoch :  0030 cost =  0.018607
Epoch :  0031 cost =  0.021008
Epoch :  0032 cost =  0.006856
Epoch : 

In [21]:
def translate(word):
    # 'P' : 모델의 자리수를 일치시키기 위해 그냥 넣음 (다른 의미 x )
    seq_data = [word,'P' * len(word)] # word,PPPP
    input_batch, output_batch, target_batch = make_batch([seq_data])
    # 1x5(S또는E 추가)x41
    prediction = tf.argmax(model,2)  # 면, 행, 열 중에서 열 중심  # 6x 3x 41 중 41에서 max값
    result = sess.run(prediction,
                     feed_dict={enc_input : input_batch,
                               dec_input : output_batch,
                               targets : target_batch})
    decoded = [char_arr[i] for i in result[0]]   # 5에서 하나씩 뺌
    end = decoded.index('E')
    translated = ''.join(decoded[:end])

    return translated
print('word ->', translate('word'))


word -> 단어


In [ ]:
# OCR : Kakao API
# https://developers.kakao.com/docs/restapi/vision#%EB%AC%B8%EC%9E%90-%EC%9D%B8%EC%8B%9D 

In [27]:
#!pip install opencv-python

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [63]:
import json
import cv2
import requests
import sys
LIMIT_PX = 1024
LIMIT_BYTE = 1024*1024
LIMIT_BOX = 40

In [64]:
image_path = 'C:/Users/ICT01_18/hyeon/2.jpg'
cv2.imread(image_path)


array([[[224, 229, 228],
        [225, 230, 229],
        [225, 230, 229],
        ...,
        [218, 220, 221],
        [217, 219, 220],
        [217, 219, 220]],

       [[224, 229, 228],
        [225, 230, 229],
        [225, 230, 229],
        ...,
        [219, 221, 222],
        [218, 220, 221],
        [218, 220, 221]],

       [[225, 230, 229],
        [225, 230, 229],
        [226, 231, 230],
        ...,
        [220, 222, 223],
        [220, 222, 223],
        [219, 221, 222]],

       ...,

       [[230, 235, 234],
        [230, 235, 234],
        [230, 235, 234],
        ...,
        [222, 225, 229],
        [222, 225, 229],
        [222, 225, 229]],

       [[229, 234, 233],
        [229, 234, 233],
        [230, 235, 234],
        ...,
        [222, 225, 229],
        [222, 225, 229],
        [222, 225, 229]],

       [[227, 232, 231],
        [228, 233, 232],
        [229, 234, 233],
        ...,
        [223, 226, 230],
        [223, 226, 230],
        [224, 227, 231]]

In [65]:
def kakao_ocr_resize(image_path: str):  # 규격 이미지 사이즈

    image = cv2.imread(image_path)         # 자기 이미지로 변경
    height, width, _ = image.shape

    if LIMIT_PX < height or LIMIT_PX < width:
        ratio = float(LIMIT_PX) / max(height, width)
        image = cv2.resize(image, None, fx=ratio, fy=ratio)
        height, width, _ = height, width, _ = image.shape

        # api 사용전에 이미지가 resize된 경우, recognize시 resize된 결과를 사용해야함.
        image_path = "{}_resized.jpg".format(image_path)
        cv2.imwrite(image_path, image)

        return image_path
    return None

In [66]:
appkey = "fefd26bbf235e38d765571b2be471226"
def kakao_ocr_detect(image_path: str, appkey: str):
    """
    detect api request example
    :param image_path: 이미지파일 경로
    :param appkey: 카카오 앱 REST API 키
    """
    API_URL = 'https://kapi.kakao.com/v1/vision/text/detect'

    headers = {'Authorization': 'KakaoAK {}'.format(appkey)}

    image = cv2.imread(image_path)
    jpeg_image = cv2.imencode(".jpg", image)[1]
    data = jpeg_image.tobytes()

    return requests.post(API_URL, headers=headers, files={"file": data})


In [67]:
def kakao_ocr_recognize(image_path: str, boxes: list, appkey: str):

    """
    recognize api request example
    :param boxes: 감지된 영역 리스트. Canvas 좌표계: 좌상단이 (0,0) / 우상단이 (limit,0)
                    감지된 영역중 좌상단 점을 기준으로 시계방향 순서, 좌상->우상->우하->좌하
                    ex) [[[0,0],[1,0],[1,1],[0,1]], [[1,1],[2,1],[2,2],[1,2]], ...]
    :param image_path: 이미지 파일 경로
    :param appkey: 카카오 앱 REST API 키
    """
    API_URL = 'https://kapi.kakao.com/v1/vision/text/recognize'

    headers = {'Authorization': 'KakaoAK {}'.format(appkey)}

    image = cv2.imread(image_path)
    jpeg_image = cv2.imencode(".jpg", image)[1]
    data = jpeg_image.tobytes()

    return requests.post(API_URL, headers=headers, files={"file": data}, data={"boxes": json.dumps(boxes)})

In [71]:
# 자동 가계부로 사용가능

def main():
    image_path = 'C:/Users/ICT01_18/hyeon/2.jpg'
    appkey = "fefd26bbf235e38d765571b2be471226"
    if len(sys.argv) != 3:
        print("Please run with args: $ python example.py /path/to/image appkey")
    #image_path, appkey = sys.argv[1], sys.argv[2]

    resize_impath = kakao_ocr_resize(image_path)
    if resize_impath is not None:
        image_path = resize_impath
        print("원본 대신 리사이즈된 이미지를 사용합니다.")

    output = kakao_ocr_detect(image_path, appkey).json()
    print("[detect] output:\n{}\n".format(output))

    boxes = output["result"]["boxes"]
    boxes = boxes[:min(len(boxes), LIMIT_BOX)]
    output = kakao_ocr_recognize(image_path, boxes, appkey).json()
    print("[recognize] output:\n{}\n".format(json.dumps(output, sort_keys=True, indent=2)))

output = main()
output['result']


[detect] output:
{'result': {'boxes': [[[139, 23], [362, 23], [362, 52], [139, 52]], [[386, 23], [451, 23], [451, 52], [386, 52]], [[32, 86], [139, 86], [139, 113], [32, 113]], [[175, 87], [425, 87], [425, 113], [175, 113]], [[145, 94], [157, 88], [165, 103], [153, 109]], [[30, 123], [140, 123], [140, 146], [30, 146]], [[148, 125], [161, 125], [161, 141], [148, 141]], [[175, 121], [439, 121], [439, 147], [175, 147]], [[129, 243], [160, 246], [157, 275], [126, 272]], [[144, 175], [194, 175], [194, 203], [144, 203]], [[158, 218], [179, 210], [189, 235], [167, 243]], [[167, 247], [183, 247], [183, 273], [167, 273]], [[196, 244], [421, 246], [421, 271], [196, 270]], [[196, 209], [336, 209], [336, 236], [196, 236]], [[200, 178], [311, 178], [311, 203], [200, 203]], [[321, 178], [378, 178], [378, 203], [321, 203]], [[353, 209], [462, 208], [463, 237], [353, 238]], [[397, 174], [461, 174], [461, 203], [397, 203]], [[5, 211], [157, 208], [159, 276], [7, 278]], [[8, 282], [119, 282], [119, 373]

TypeError: 'NoneType' object is not subscriptable

In [70]:
sys.argv

['C:\\Users\\ICT01_18\\Anaconda3\\envs\\tf_test\\lib\\site-packages\\ipykernel_launcher.py',
 '-f',
 'C:\\Users\\ICT01_18\\AppData\\Roaming\\jupyter\\runtime\\kernel-dd707d51-cc5f-40a6-9512-6388d31d751e.json']

In [72]:
# pip install opencv-python
import json
import cv2 # computer vision tool (이미지, 동영상 처리)
import requests # html 페이지 요청
import sys
LIMIT_PX = 1024
LIMIT_BYTE = 1028*1024
LIMIT_BOX = 40
# OCR 절차
# 이미지 사이즈 조정
# 글씨를 둘러싼 Box를 detection

def kakao_ocr_resize(image_path: str):
    image = cv2.imread('2.jpg')
    height, width, _ = image.shape
    if LIMIT_PX < height or LIMIT_PX < width:
        ratio = float(LIMIT_PX) / max(heigth, width)
        image = cv2.resize(image, None, fx=ratio, fy=ratio)
        height, width, _ = height, width, _ = image.shape
        image_path = "{}_resize.jpg".format('2')
        cv2.imwrite(image_path, image)
        return image_path
    return None

# 사용법
# open api 함수를 원격호출 (remote call)
def kakao_ocr_detect(image_path: str, appkey: str):
    API_URL ='https://kapi.kakao.com/v1/vision/text/detect'
    headers = {'Authorization': 'KakaoAK {}'.format('7d278c16a4138b64cf7e96eb6fb432a4')}
    image = cv2.imread('2.jpg') # 이미지 로드
    # naver는 jpg만, gif 애니메이션 지원, png 투명도 지원
    jpeg_image = cv2.imencode('.jpg', image)[1]
    data = jpeg_image.tobytes() # 네크워크 전송 -> serialization
    # 네트워크일 때 => 데이터가 시리얼로 가야 함 (모뎀)
    # 8개 라인으로 구성, 4개 회선만 사용 (1개 전송, 1개 수신)
    # 네트워크를 통해 함수 호출 (시간) : 비동기 방식
    # 동기 방식(호출 후 돌아올 때까지 기다림), 비동기 방식(다른 작업 중 -event 받아서 작업)
    # 대표적인 비동기 통신 방식이 AJAX : 웹 브라우저 내부적 처리
    # 인터넷 표준 데이터 형식 json
    return requests.post(API_URL, headers=headers, files={"file": data})

def kakao_ocr_recognize(image_path: str, boxes: list, appkey: str):
    API_URL = 'https://kapi.kakao.com/v1/vision/text/recognize'
    headers = {'Authorization': 'KakaoAK {}'.format('7d278c16a4138b64cf7e96eb6fb432a4')}
    image = cv2.imread('2.jpg')
    jpeg_image = cv2.imencode('.jpg', image)[1]
    data = jpeg_image.tobytes()
    # 인식은 BOX로 텍스트 구역을 확인하고, recognize 인식한다.
    return requests.post(API_URL, headers=headers, files={"file": data}, data={"boxes": json.dumps(boxes)})

def main():
    image_path, appkey = sys.argv[1], sys.argv[2]
    resize_impath = kakao_ocr_resize(image_path)
    output = kakao_ocr_detect(image_path, appkey).json()
    boxes = output['result']['boxes']
    boxes = boxes[:min(len(boxes), LIMIT_BOX)]
    output = kakao_ocr_recognize(image_path, boxes, appkey).json()
    return output
output = main()
output['result']

{'recognition_words': ['신용카드매출',
  '내 역',
  '카드번호',
  '9440-0382-1745-****',
  ':',
  '카 드 사',
  ':',
  '비씨카드/IBK비씨제크',
  '호',
  '[IBK',
  ':',
  ':',
  '94400382****9857',
  '2013-05-26',
  '비씨체크',
  '신용',
  '14:09:41',
  '승인]',
  '슭밑윕싫',
  '출 국 비',
  'ø9YYU460',
  '알무개월 :00',
  '16,000',
  '781299839',
  '비씨카드',
  '사업자',
  '번호',
  '616-28-83377',
  '2013-05-26',
  '14:09',
  'CASHIER',
  '직원']}

In [73]:
sys.argv

['C:\\Users\\ICT01_18\\Anaconda3\\envs\\tf_test\\lib\\site-packages\\ipykernel_launcher.py',
 '-f',
 'C:\\Users\\ICT01_18\\AppData\\Roaming\\jupyter\\runtime\\kernel-dd707d51-cc5f-40a6-9512-6388d31d751e.json']

In [15]:
# 영상 읽어오기
# 배경 제거 전
import cv2
import numpy as np
cap = cv2.VideoCapture('corrupted_video.mp4')   # 위 경로에에서 video 캡처 명령
if (cap.isOpened() == False) : # 파일이 없거나 장치가 없을 때
    print("Error 발생")

while(cap.isOpened()) :
    ret, frame = cap.read()  
    # 1개의 frame을 읽어옴( 동영상은 ntsc 방식으로 초당 29.7프레임)
    
    if ret == True:
        cv2.imshow('Frame', frame)
        if cv2.waitKey(25) & 0xFF == ord('q'): # q를 누르면 종료
            break
    else : 
        break
cap.release()
cv2.destroyAllWindows()


In [16]:
# 영상 읽어오기
# 배경 제거
import cv2
import numpy as np
history = 30
cap = cv2.VideoCapture('corrupted_video.mp4')   # 위 경로에에서 video 캡처 명령
if (cap.isOpened() == False) : # 파일이 없거나 장치가 없을 때
    print("Error 발생")

fgbg = cv2.createBackgroundSubtractorMOG2()  # 배경 필터(background filter)
while(cap.isOpened()) :
    ret, frame = cap.read()  
    # 1개의 frame을 읽어옴( 동영상은 ntsc 방식으로 초당 29.7프레임)
    
    if ret == True:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        fgmask = fgbg.apply(gray, learningRate = 1.0/history)
        cv2.imshow('Frame', fgmask)
        if cv2.waitKey(25) & 0xFF == ord('q'): # q를 누르면 종료
            break
    else : 
        break
cap.release()
cv2.destroyAllWindows()





In [17]:
# 영상 저장하기
# 동영상 데이터의 전처리 => AI model에 훈련 => 실시간 인식
# 
import numpy as np
import cv2 #computer vision ==> 사운드가 안됨

cap = cv2.VideoCapture('corrupted_video.mp4') 
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0 , (640,480)) # 윈도우 표준 형식 avi
while(cap.isOpened()) :
    ret, frame = cap.read()   # embeded 장치에 활용(차선인식 등)
    if ret == True:
        frame = cv2.flip(frame,0)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # 고속으로 가능( 흑백 처리- 생략 가능 )
        out.write(frame)
        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else :
        break

cap.release()
out.release()
cv2.destroyAllWindows()

In [1]:
# AE(Auto Encoder )
# denoising autoencoder
# 원본데이터, noise가 있는 원본 데이터 
# 학습은 noise가 있는 학습 데이터 => 가중치
# 노이즈가 있는 데이터를 원본데이터로 출력함
%matplotlib inline
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import tensorflow as tf


def vis(images, save_name):
    dim = images.shape[0]
    n_image_rows = int(np.ceil(np.sqrt(dim)))
    n_image_cols = int(np.ceil(dim* 1.0/n_image_rows))
    gs = gridspec.GridSpec(n_image_rows, n_image_cols, top = 1., bottom= 0.,
                          right = 1., left = 0., hspace = 0., wspace = 0.)
    for g, count in zip(gs, range(int(dim))):
        ax = plt.subplot(g)
        ax.imshow(images[count,:].reshape((28,28)))
        ax.set_xticks([])
        ax.set_yticks([])
    plt.savefig(save_name + '_vis.png')
    plt.show()
mnist_width = 28
n_visible = mnist_width * mnist_width
n_hidden = 500

C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\site-packages

In [2]:
corruption_level = 0.3
# 784
X = tf.placeholder('float', [None, n_visible], name = 'X')
mask = tf.placeholder('float', [None, n_visible], name = 'mask')

W_init_max = 4 * np.sqrt(6./ (n_visible + n_hidden))
# 균등분포에서 데이터 생성( 가중치 초기화)
W_init = tf.random_uniform(shape = [n_visible, n_hidden],
                          minval = -W_init_max, maxval = W_init_max)

W = tf.Variable(W_init, name = 'W')
b = tf.Variable(tf.zeros([n_hidden]), name = 'b')




In [3]:
W_prime = tf.transpose(W) # 자기자신의 전치
b_prime = tf.Variable(tf.zeros([n_visible]), name = 'b_prime')
def model(X, mask, W,b, W_prime, b_prime):
    tilde_X  = mask * X  # 행렬 요소곱
    Y = tf.nn.sigmoid(tf.matmul(tilde_X,W) + b)
    Z = tf.nn.sigmoid(tf.matmul(Y, W_prime) + b_prime)
    return Z

Z = model(X,mask, W,b , W_prime , b_prime)

# 가중치에 이미지의 특성이 추출됨




In [8]:
cost = tf.reduce_mean(tf.pow(X-Z, 2))  # X : 원본데이터(깔끔한 이미지) / Z : 예측된 데이터( 자기자신(잡티가 있는 이미지))
# 오토인코더는 인풋과 아웃풋이 모두 자기 자신
# 비지도 학습 => 데이터의 특징을 추출함
#  인풋데이터ㅇ에 노이즈를 부여하여 이미지를 부분 부분 없앰 => 학습하면 원래의 데이터가 나옴


train_op = tf.train.GradientDescentOptimizer(0.02).minimize(cost)
predict_op = Z

mnist = input_data.read_data_sets('MNIST_data/', one_hot = True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

NameError: name 'Z' is not defined

In [ ]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(100) :
        for start, end in zip(range(0, len(trX), 128),
                             range(128, len(trX)+1, 128)):
            input_ = trX[start:end]
            # 이항분포로 마스크 생성( 원본이미지에 노이즈를 주기 위해 ) => 
            mask_up = np.random.binomial(1,1-corruption_level,
                                        input_.shape)
            sess.run(train_op, feed_dict = {X:input_, mask : mask_up})
        mask_up = np.random.binomial(1,1 - corruption_level,
                                    teX.shape)
        print(i, sess.run(cost, feed_dict={X : teX, mask : mask_up}))
    mask_np = np.random.binomial(1,1-corruption_level,
                                teX[:100].shape)
    
    # 예측할 때 이미지에 마스크
    predicted_imgs = sess.run(predict_op, feed_dict={X:teX[:100],
                                                    mask : mask_up})
    input_imgs = teX[:100]
vis(predicted_img, 'pred') # 예측된 이미지
vis(input_imgs, 'in')  # 입력 이미지


0 0.2588214
1 0.22730303


In [ ]:
# Vaiational autoencoder ( VAE)
# generative model 생성모델
#  * 판별모델(discrimitive model) : 선을 그어서 분류를 하는 모델( tree)
#  * 생성모델 : 확률을 찾아냄-> 평균, 분산 고려( 분포 )
#     - 잠재변수latent variables : 이미지를 보고 이미지의 특징만 잡아냄(키, 성별 등)
#                                 => 특징을 바탕으로 새로운 이미지 만들어 낼 수 있음(생성모델)
#                                - 특징을 뽑아

In [14]:
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
def vis(images, save_name):
    dim = images.shape[0]
    n_image_rows = int(np.ceil(np.sqrt(dim)))
    n_image_cols = int(np.ceil(dim* 1.0/n_image_rows))
    gs = gridspec.GridSpec(n_image_rows, n_image_cols, top = 1., bottom= 0.,
                          right = 1., left = 0., hspace = 0., wspace = 0.)
    for g, count in zip(gs, range(int(dim))):
        ax = plt.subplot(g)
        ax.imshow(images[count,:].reshape((28,28)))
        ax.set_xticks([])
        ax.set_yticks([])
    plt.savefig(save_name + '_vis.png')
    plt.show()
mnist_width = 28
n_visible = mnist_width * mnist_width
n_hidden = 500

corruption_level = 0.3
X = tf.placeholder("float", [None, n_visible], name 
                   
                   = 'X')
mask = tf.placeholder("float", [None, n_visible], name = 'mask')
W_init_max = 4 * np.sqrt(6. / (n_visible+ n_hidden))
W_init = tf.random_uniform(shape = [n_visible, n_hidden], minval = -W_init_max,
                          maxval = W_init_max)
W = tf.Variable(W_init, name ='W')
b = tf.Variable(tf.zeros([n_hidden]), name = 'b')

W_prime = tf.transpose(W)
b_prime = tf.Variable(tf.zeros([n_visible]), name = 'b_prime')
def model(X, mask, W, b, W_prime, b_prime):
    tilde_X = mask * X
    Y = tf.nn.sigmoid(tf.matmul(tilde_X, W)+ b)
    Z = tf.nn.sigmoid(tf.matmul(Y, W_prime) + b_prime)
    return Z
Z = model(X, mask, W, b, W_prime, b_prime)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(100):
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX)+1, 128 )):
            input_= trX[start:end]
            mask_np = np.random.binomial(1,1-corruption_level, input_.shape)
            sess.run(train_op, feed_dict = {X:input_, mask:mask_np})
        mask_np = np.random.binomial(1,1 - corruption_level, teX.shape)
        print(i, sess.run(cost, feed_dict = {X:teX, mask:mask_np}))
    mask_np = np.random.binomial(1, 1 -corruption_level, teX[:100].shape)
    predicted_imgs =sess.run(predict_op, feed_dict = {X: teX[:100], mask:mask_np})
    input_imgs = teX[:100]
vis(predicted_imgs, 'pred')
vis(input_imgs, 'in')

InvalidArgumentError: You must feed a value for placeholder tensor 'mask_4' with dtype float and shape [?,784]
	 [[node mask_4 (defined at <ipython-input-10-61a6aa8cc4f9>:4) ]]

Original stack trace for 'mask_4':
  File "C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\site-packages\ipykernel\kernelapp.py", line 583, in start
    self.io_loop.start()
  File "C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\site-packages\tornado\platform\asyncio.py", line 148, in start
    self.a

In [19]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.framework import ops
import numpy as np
ops.reset_default_graph()
train, test = tf.keras.datasets.mnist.load_data()
train_x , train_y = train
from functools import partial
mnist = input_data.read_data_sets('MNIST_data/', one_hot = True)

n_inputs = 28*28
n_hidden1 = 500
n_hidden2 = 500
n_hidden3 = 20
n_hidden4 = n_hidden2
n_hidden5 = n_hidden1
n_outputs = n_inputs  # 입력이 출력으로
learning_rate = 0.001
# without scaling 초기화( 스케일링 없이 초기화 )
initializer = tf.contrib.layers.variance_scaling_initializer()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [20]:
my_dense_layer = partial(   # 매개변수 일부를 고정 => 상수취급
    tf.layers.dense,
    activation = tf.nn.elu,  # Exponential linear Unit ( -1을 고려한 relu)
    kernel_initializer = initializer)
X = tf.placeholder(tf.float32, [None, n_inputs])
hidden1 = my_dense_layer(X, n_hidden1)
hidden2 = my_dense_layer(hidden1, n_hidden2)
hidden3_mean = my_dense_layer(hidden2, n_hidden3, activation = None)
hidden3_sigma = my_dense_layer(hidden2, n_hidden3, activation = None)
noise = tf.random_normal(tf.shape(hidden3_sigma), dtype = tf.float32)
# 평균 시그마(분산) 노이즈  => 변동된 특성을 추출하기 위함 ( 요소곱 )
hidden3 = hidden3_mean + hidden3_sigma * noise

hidden4 = my_dense_layer(hidden3, n_hidden4)
hidden5 = my_dense_layer(hidden4, n_hidden5)
logits = my_dense_layer(hidden5, n_outputs, activation= None)
outputs = tf.sigmoid(logits)

In [21]:
xentropy = tf.nn.sigmoid_cross_entropy_with_logits(labels = X, logits = logits)

# 재구축 손실
reconstruction_loss = tf.reduce_sum(xentropy)

eps = 1e-10 # 앱실론(epsilon)상수 : 로그의 성질 -- 0이면 무한대 => 무한대로 가는 것을 막기 위함
# 해당 앱실론 값 이하의 차이면 같은 것으로 간주

# cost function : log liklihood (우도 확률)을 고려한 식 
# 지연손실(확률 분포 곡선 )
latent_loss = 0.5 * tf.reduce_mean( 
    tf.square(hidden3_sigma) + tf.square(hidden3_mean)- 1 - tf.log(eps + tf.square(hidden3_sigma)))

loss = reconstruction_loss + latent_loss
optimizer=  tf.train.AdamOptimizer(learning_rate= learning_rate)
training_op= optimizer.minimize(loss)
init = tf.global_variables_initializer()

In [22]:
import sys
n_digis = 60
n_epochs = 50
batch_size = 150
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs) :
        n_batches = mnist.train.num_examples // batch_size
        for iteration in range(n_batches):
            print("\r{}%".format(100 * iteration // n_batches),
                 end = '')
            sys.stdout.flush()
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict = {X:X_batch})
        loss_val , reconstruction_loss_val, latent_loss_val = sess.run([loss, reconstruction_loss , latent_loss], feed_dict={X:X_batch})
        print("\r{}".format(epoch), 'Train total loss: ', loss_val, '\tReconstruction loss: ',
             reconstruction_loss_val, '\tLatent loss : ', latent_loss_val)
        codings_rnd = np.random.normal(size = [n_digis, n_hidden3])
        outputs_val = outputs.eval(feed_dict={hidden3: codings_rnd})
        
# 가중치를 통과한 데이터 ( tset ) : 모델 완성
# PCA처럼 비지도학습의 특성으로 추출된 데이터 ( 분류와 예측이 잘되기 위해서 함 )

0 Train total loss:  12777.681 	Reconstruction loss:  12773.199 	Latent loss :  4.481745
1 Train total loss:  12188.6455 	Reconstruction loss:  12183.971 	Latent loss :  4.6745887
2 Train total loss:  11055.033 	Reconstruction loss:  11050.315 	Latent loss :  4.7176256
3 Train total loss:  10117.8125 	Reconstruction loss:  10112.883 	Latent loss :  4.9295235
49% Train total loss:  10697.864 	Reconstruction loss:  10692.973 	Latent loss :  4.8915434
5 Train total loss:  10431.455 	Reconstruction loss:  10426.429 	Latent loss :  5.0265617
6 Train total loss:  10381.528 	Reconstruction loss:  10376.626 	Latent loss :  4.9027863
7 Train total loss:  10219.271 	Reconstruction loss:  10214.192 	Latent loss :  5.078694
8 Train total loss:  9916.226 	Reconstruction loss:  9911.135 	Latent loss :  5.091222
9 %Train total loss:  9235.441 	Reconstruction loss:  9230.321 	Latent loss :  5.120447
10 Train total loss:  9755.961 	Reconstruction loss:  9750.6875 	Latent loss :  5.273396
11 Train total